# Using a ✨Secret API✨ to scrape ATL Hawks line-ups from NBA.com

In [122]:
import pandas as pd
import requests


## 5-player line-ups

By copying the cURL of 'leaguedashlineups' under the 'Network' tab of the inspection page on NBA.com's lineups for the Hawks, you can use [Convert cURL](https://curlconverter.com/#) to access the proper headers and parameters for the NBA.com API. Below, you'll see the output of the parameters for the Hawks' 5-player line-ups for the 2021-22 season.
[Data source](https://www.nba.com/stats/lineups/traditional/?Season=2021-22&SeasonType=Regular%20Season&TeamID=1610612737)

**Step 1:** Grab the data using requests library 

In [123]:
import requests

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'sec-ch-ua-platform': '"macOS"',
    'Origin': 'https://www.nba.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.nba.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

params = (
    ('Conference', ''),
    ('DateFrom', ''),
    ('DateTo', ''),
    ('Division', ''),
    ('GameID', ''),
    ('GameSegment', ''),
    ('GroupQuantity', '5'),
    ('LastNGames', '0'),
    ('LeagueID', '00'),
    ('Location', ''),
    ('MeasureType', 'Base'),
    ('Month', '0'),
    ('OpponentTeamID', '0'),
    ('Outcome', ''),
    ('PORound', '0'),
    ('PaceAdjust', 'N'),
    ('PerMode', 'PerGame'),
    ('Period', '0'),
    ('PlusMinus', 'N'),
    ('Rank', 'N'),
    ('Season', '2021-22'),
    ('SeasonSegment', ''),
    ('SeasonType', 'Regular Season'),
    ('ShotClockRange', ''),
    ('TeamID', '1610612737'),
    ('VsConference', ''),
    ('VsDivision', ''),
)

response = requests.get('https://stats.nba.com/stats/leaguedashlineups', headers=headers, params=params)
response.json().keys()

dict_keys(['resource', 'parameters', 'resultSets'])

In [124]:
#response.json()

**Step 2:** Organize the data
* This is a little tricky at first glance. The API gives us a list of keys and a list of lists holding our rows. 
* To organize, we isolate both entities into "keys" (a list of keys) and "values_list" (a list of lists, holding values)
* First, we turn "values_list" (a list of lists) into a dataframe. 
* Then, set df.columns = keys
* And done! 

In [126]:
import pandas as pd 
keys = response.json()['resultSets'][0]['headers']
values_list = response.json()['resultSets'][0]['rowSet']

# Making sure we have the right number of rows
len(values_list) 


346

In [127]:
df = pd.DataFrame(values_list)

df.columns = keys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-203991-203992-1626153-1628381-1629629-,C. Capela - B. Bogdanovic - D. Wright - J. Col...,1610612737,ATL,1,0,1,0.000,24.0,15.0,45.0,0.333,5.0,17.0,0.294,6.0,11.0,0.545,6.0,17.0,23.0,12.0,5.0,5.0,3.0,4.0,6.0,8.0,41.0,-16.0,123,204,119,204,1,2,1,212,1,1,157,3,1,171,2,2,2,1,342,1,1,346,345,2,2,346
1,Lineups,-203991-203992-1629027-1629629-1630602-,C. Capela - B. Bogdanovic - T. Young - C. Redd...,1610612737,ATL,1,0,1,0.000,19.1,16.0,39.0,0.410,5.0,13.0,0.385,5.0,5.0,1.000,6.0,18.0,24.0,8.0,6.0,2.0,3.0,3.0,4.0,5.0,42.0,-6.0,123,204,119,204,2,1,2,183,1,2,121,8,19,1,2,1,1,4,345,5,1,343,324,9,1,328
2,Lineups,-203991-203992-1628381-1629027-1629631-,C. Capela - B. Bogdanovic - J. Collins - T. Yo...,1610612737,ATL,14,5,9,0.357,16.0,15.0,29.9,0.502,3.6,8.6,0.425,4.5,6.2,0.724,3.6,11.5,15.1,8.6,4.1,2.0,1.2,1.5,5.8,6.6,38.1,1.0,1,7,346,180,3,2,3,105,7,10,109,13,7,150,9,6,5,2,339,5,18,332,344,4,3,99
3,Lineups,-203991-1627789-1628381-1628989-1629027-,C. Capela - T. Luwawu-Cabarrot - J. Collins - ...,1610612737,ATL,10,3,7,0.300,15.1,13.8,29.6,0.466,4.6,11.4,0.404,5.5,7.0,0.786,4.8,12.1,16.9,8.4,4.2,2.0,2.0,2.1,5.7,5.4,37.7,3.1,7,25,343,195,4,4,4,156,6,3,112,7,5,136,4,5,4,3,340,5,5,342,343,8,5,44
4,Lineups,-101150-201568-203992-1628989-1630168-,L. Williams - D. Gallinari - B. Bogdanovic - K...,1610612737,ATL,1,0,1,0.000,14.9,12.0,25.0,0.480,5.0,11.0,0.455,9.0,11.0,0.818,2.0,13.0,15.0,7.0,2.0,0.0,2.0,1.0,4.0,7.0,38.0,2.0,123,204,119,204,5,7,6,149,1,4,99,1,1,128,20,3,6,8,300,137,5,293,324,3,4,73


**Step 3:** Cleaning the data

Luckily this data doesn't require too much cleaning. Here, we are changing the column names to make them easier to work with and dropping a few extraneous columns that we don't need for this analysis. 

**NOTE:** The "hidden" API actually gives us a few more units of analysis than the public-facing page, ranking each line-up by each metric (such as offensive rebound rank or assist rank). This is very exciting for my purposes in this analysis and for future uses of this data! 

In [128]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [129]:
df = df.drop(['group_id', 'group_set', 'team_id'], 1)

/var/folders/qx/nkh2pq6x3_193757ddxvrcf40000gp/T/ipykernel_39134/3071107983.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['group_id', 'group_set', 'team_id'], 1)


In [130]:
#df.to_csv('five_player_lineups_2122.csv', index=False)

## 3-player line-ups
Using the same method as above to grab the 3-player line-ups for the Hawks current 2021-22 season.
[Data source](https://www.nba.com/stats/lineups/traditional/?Season=2021-22&SeasonType=Regular%20Season&TeamID=1610612737&GroupQuantity=3)

**Step 1:** Pull out Data

In [144]:
import requests

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'sec-ch-ua-platform': '"macOS"',
    'Origin': 'https://www.nba.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.nba.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

params = (
    ('Conference', ''),
    ('DateFrom', ''),
    ('DateTo', ''),
    ('Division', ''),
    ('GameID', ''),
    ('GameSegment', ''),
    ('GroupQuantity', '3'),
    ('LastNGames', '0'),
    ('LeagueID', '00'),
    ('Location', ''),
    ('MeasureType', 'Base'),
    ('Month', '0'),
    ('OpponentTeamID', '0'),
    ('Outcome', ''),
    ('PORound', '0'),
    ('PaceAdjust', 'N'),
    ('PerMode', 'PerGame'),
    ('Period', '0'),
    ('PlusMinus', 'N'),
    ('Rank', 'N'),
    ('Season', '2021-22'),
    ('SeasonSegment', ''),
    ('SeasonType', 'Regular Season'),
    ('ShotClockRange', ''),
    ('TeamID', '1610612737'),
    ('VsConference', ''),
    ('VsDivision', ''),
)

response = requests.get('https://stats.nba.com/stats/leaguedashlineups', headers=headers, params=params)
response.json().keys()

dict_keys(['resource', 'parameters', 'resultSets'])

**Step 2:** Organize data

In [145]:
import pandas as pd 
keys = response.json()['resultSets'][0]['headers']
values_list = response.json()['resultSets'][0]['rowSet']

In [146]:
# Double checking that we've grabbed all the rows 
len(values_list)

657

In [147]:
df = pd.DataFrame(values_list)

df.columns = keys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-203991-1628411-1629027-,C. Capela - W. Iwundu - T. Young,1610612737,ATL,2,1,1,0.500,24.5,22.5,49.5,0.455,7.5,17.5,0.429,11.0,12.0,0.917,7.5,15.5,23.0,11.5,2.0,4.0,2.0,4.5,9.0,10.5,63.5,4.0,311,257,144,240,1,1,1,303,1,1,193,2,2,168,3,6,4,3,488,1,12,655,652,1,1,76
1,Lineups,-203992-1629027-1630602-,B. Bogdanovic - T. Young - C. Brown Jr.,1610612737,ATL,1,0,1,0.000,23.8,21.0,48.0,0.438,6.0,16.0,0.375,5.0,5.0,1.000,7.0,19.0,26.0,11.0,6.0,2.0,3.0,3.0,6.0,5.0,53.0,-11.0,408,490,144,490,2,2,2,330,2,4,295,46,81,1,4,1,1,5,647,28,2,643,625,49,5,653
2,Lineups,-101150-203991-1628411-,L. Williams - C. Capela - W. Iwundu,1610612737,ATL,1,1,0,1.000,23.3,20.0,48.0,0.417,6.0,17.0,0.353,10.0,12.0,0.833,8.0,14.0,22.0,8.0,1.0,2.0,3.0,5.0,10.0,8.0,56.0,-1.0,408,257,1,1,4,4,2,385,2,2,345,3,2,270,1,13,5,24,257,28,2,656,656,7,3,386
3,Lineups,-101150-1628411-1629027-,L. Williams - W. Iwundu - T. Young,1610612737,ATL,1,1,0,1.000,23.3,20.0,48.0,0.417,6.0,17.0,0.353,13.0,15.0,0.867,8.0,14.0,22.0,8.0,1.0,2.0,3.0,5.0,10.0,9.0,59.0,2.0,408,257,1,1,3,4,2,385,2,2,345,1,1,220,1,13,5,24,257,28,2,656,656,4,2,134
4,Lineups,-1628381-1629027-1629631-,J. Collins - T. Young - D. Hunter,1610612737,ATL,26,12,14,0.462,22.9,20.8,42.5,0.490,5.2,14.5,0.361,7.5,9.8,0.765,4.8,15.8,20.6,12.7,6.0,3.0,2.2,2.2,9.4,10.0,54.3,-0.6,18,27,639,353,5,3,5,237,8,5,328,15,10,366,13,5,9,1,655,2,10,641,655,2,4,372


**Step 3:** Clean data and export to CSV

In [148]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df = df.drop(['group_id', 'group_set', 'team_id'], 1)
df.head()

/var/folders/qx/nkh2pq6x3_193757ddxvrcf40000gp/T/ipykernel_39134/1215226637.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['group_id', 'group_set', 'team_id'], 1)


,group_name,team_abbreviation,gp,w,l,w_pct,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,pts,plus_minus,gp_rank,w_rank,l_rank,w_pct_rank,min_rank,fgm_rank,fga_rank,fg_pct_rank,fg3m_rank,fg3a_rank,fg3_pct_rank,ftm_rank,fta_rank,ft_pct_rank,oreb_rank,dreb_rank,reb_rank,ast_rank,tov_rank,stl_rank,blk_rank,blka_rank,pf_rank,pfd_rank,pts_rank,plus_minus_rank
0,C. Capela - W. Iwundu - T. Young,ATL,2,1,1,0.500,24.5,22.5,49.5,0.455,7.5,17.5,0.429,11.0,12.0,0.917,7.5,15.5,23.0,11.5,2.0,4.0,2.0,4.5,9.0,10.5,63.5,4.0,311,257,144,240,1,1,1,303,1,1,193,2,2,168,3,6,4,3,488,1,12,655,652,1,1,76
1,B. Bogdanovic - T. Young - C. Brown Jr.,ATL,1,0,1,0.000,23.8,21.0,48.0,0.438,6.0,16.0,0.375,5.0,5.0,1.000,7.0,19.0,26.0,11.0,6.0,2.0,3.0,3.0,6.0,5.0,53.0,-11.0,408,490,144,490,2,2,2,330,2,4,295,46,81,1,4,1,1,5,647,28,2,643,625,49,5,653
2,L. Williams - C. Capela - W. Iwundu,ATL,1,1,0,1.000,23.3,20.0,48.0,0.417,6.0,17.0,0.353,10.0,12.0,0.833,8.0,14.0,22.0,8.0,1.0,2.0,3.0,5.0,10.0,8.0,56.0,-1.0,408,257,1,1,4,4,2,385,2,2,345,3,2,270,1,13,5,24,257,28,2,656,656,7,3,386
3,L. Williams - W. Iwundu - T. Young,ATL,1,1,0,1.000,23.3,20.0,48.0,0.417,6.0,17.0,0.353,13.0,15.0,0.867,8.0,14.0,22.0,8.0,1.0,2.0,3.0,5.0,10.0,9.0,59.0,2.0,408,257,1,1,3,4,2,385,2,2,345,1,1,220,1,13,5,24,257,28,2,656,656,4,2,134
4,J. Collins - T. Young - D. Hunter,ATL,26,12,14,0.462,22.9,20.8,42.5,0.490,5.2,14.5,0.361,7.5,9.8,0.765,4.8,15.8,20.6,12.7,6.0,3.0,2.2,2.2,9.4,10.0,54.3,-0.6,18,27,639,353,5,3,5,237,8,5,328,15,10,366,13,5,9,1,655,2,10,641,655,2,4,372


In [149]:
#df.to_csv('three_player_lineups_2122.csv', index=False)

## 2-player line-ups
Last but not least, repeating one more time to grab 2-player pairs for the Hawks 2021-22 season.
[Data source](https://www.nba.com/stats/lineups/traditional/?Season=2021-22&SeasonType=Regular%20Season&TeamID=1610612737&GroupQuantity=2)

**Step 1:** Pull out Data

In [137]:
import requests

headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'sec-ch-ua-platform': '"macOS"',
    'Origin': 'https://www.nba.com',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.nba.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

params = (
    ('Conference', ''),
    ('DateFrom', ''),
    ('DateTo', ''),
    ('Division', ''),
    ('GameID', ''),
    ('GameSegment', ''),
    ('GroupQuantity', '2'),
    ('LastNGames', '0'),
    ('LeagueID', '00'),
    ('Location', ''),
    ('MeasureType', 'Base'),
    ('Month', '0'),
    ('OpponentTeamID', '0'),
    ('Outcome', ''),
    ('PORound', '0'),
    ('PaceAdjust', 'N'),
    ('PerMode', 'PerGame'),
    ('Period', '0'),
    ('PlusMinus', 'N'),
    ('Rank', 'N'),
    ('Season', '2021-22'),
    ('SeasonSegment', ''),
    ('SeasonType', 'Regular Season'),
    ('ShotClockRange', ''),
    ('TeamID', '1610612737'),
    ('VsConference', ''),
    ('VsDivision', ''),
)

response = requests.get('https://stats.nba.com/stats/leaguedashlineups', headers=headers, params=params)
#response.json()


In [138]:
response.json().keys()

dict_keys(['resource', 'parameters', 'resultSets'])

**Step 2:** Organize the data

In [139]:
keys = response.json()['resultSets'][0]['headers']
values_list = response.json()['resultSets'][0]['rowSet']

In [140]:
df = pd.DataFrame(values_list)
df.columns = keys
df.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Lineups,-1628381-1629027-,J. Collins - T. Young,1610612737,ATL,46,23,23,0.500,27.9,25.1,51.8,0.484,7.0,18.8,0.370,9.7,12.4,0.784,5.8,20.0,25.8,15.1,7.1,3.8,2.8,2.7,10.8,11.9,66.8,1.5,3,3,199,75,1,1,2,68,4,2,111,7,5,123,7,1,2,1,201,3,3,196,200,2,1,57
1,Lineups,-1629027-1629631-,T. Young - D. Hunter,1610612737,ATL,29,14,15,0.483,26.9,23.9,49.2,0.486,6.6,17.6,0.377,9.8,12.4,0.794,5.4,18.4,23.9,14.0,7.2,3.7,2.5,2.7,10.9,12.0,64.3,-0.5,24,25,178,102,2,2,5,65,5,4,101,6,4,116,9,3,3,2,202,5,5,195,201,1,5,121
2,Lineups,-101150-1628411-,L. Williams - W. Iwundu,1610612737,ATL,1,1,0,1.000,26.5,22.0,53.0,0.415,7.0,20.0,0.350,15.0,17.0,0.882,8.0,15.0,23.0,9.0,1.0,2.0,3.0,5.0,13.0,10.0,66.0,1.0,165,122,1,1,3,6,1,141,3,1,138,1,1,47,1,16,7,21,33,29,1,204,203,7,2,64
3,Lineups,-203991-1629027-,C. Capela - T. Young,1610612737,ATL,47,24,23,0.511,24.9,23.1,47.6,0.486,6.5,16.9,0.384,8.4,10.8,0.774,5.9,17.9,23.9,13.3,6.2,3.7,2.3,2.6,9.8,10.1,61.1,1.1,2,1,199,74,5,3,7,66,6,6,94,12,9,131,6,5,4,3,199,4,8,194,199,5,6,63
4,Lineups,-203991-1628411-,C. Capela - W. Iwundu,1610612737,ATL,2,1,1,0.500,24.9,23.0,50.5,0.455,8.0,18.5,0.432,11.0,12.0,0.917,7.5,15.5,23.0,12.0,2.0,4.0,2.0,4.5,9.0,10.5,65.0,4.5,135,122,30,75,4,4,3,92,1,3,51,4,6,31,2,14,7,7,103,1,13,202,195,4,3,18


**Step 3:** Clean the data and export to final csv

In [141]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [142]:
df = df.drop(['group_set', 'group_id', 'team_id'], 1)

/var/folders/qx/nkh2pq6x3_193757ddxvrcf40000gp/T/ipykernel_39134/1508230606.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(['group_set', 'group_id', 'team_id'], 1)


In [143]:
#df.to_csv('two_player_lineups_2122.csv', index=False)